In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Make sure devision by zero throws errors in stead of warnings.
np.seterr(divide='raise', over='raise', under='raise', invalid='raise')

# Evaluatie

We hebben nu gezien hoe we user ratings kunnen voorspellen. We hebben verschillende aanpakken gezien: _user-based_ en _item-based_. We hebben verschillende _similarity-maten_ gezien: _cosine similarity_ en _eucldidean similarity_. Wat hebben we hier nu aan? Wat is de beste aanpak?

Wat de beste aanpak is valt geen eenduidig antwoord op te geven. Dit hangt af van de aard van de data. We kunnen dus wel gaan onderzoeken wat het beste werkt voor een specifieke dataset. In deze opdracht gaan we kijken naar wat het beste werkt voor de MovieLens dataset.

Wat we willen weten is, hoe goed zijn de voorspellingen van de verschillende aanpakken  voor de MovieLens dataset? We gebruiken hier weer de MovieLens dataset, maar dit keer gebruiken we een iets grotere subset.

### De strategie:

Een veelgebruikte strategie is het opsplitsen van de data in een _training set_ en een _test set_. Op basis van de _training set_ leren we de overeenkomsten, waarmee we vervolgens voorspellingen kunnen doen voor de _test set_. Deze scheiding is belangrijk omdat zo de _test set_ onbekende data blijft, deze hebben we niet meegenomen in het leren van de overeenkomsten. Dus kunnen we testen of de voorspellingen op basis van de overeenkomsten uit de _training set_ ook werken voor nieuwe / onbekende data.

Hiervoor doorlopen we straks de volgende stappen:

* We splitsen de data vervolgens op in twee subsets: een _test set_ en een _training set_.
* Op basis van de _training set_ maken we een similarity matrix.
* Daaruit voorspellen we de ratings voor de film/user combinaties in de _test set_.
* Vervolgens vergelijken we de voorspelde ratings met de echte ratings uit de _test set_.

### Importeer alles uit deel 1

De code hieronder importeert alle functies die je in deel 1 hebt geschreven. 

In [ ]:
%%capture
%run "collaborative_filtering_1.ipynb"

Je zal zien dat bij het gebruiken van een grotere dataset er altijd situaties zijn waar je nog geen rekening mee had gehouden. Bijvoorbeeld, welke voorspelling geef je als er geen bepaalde film of gebruiker geen neighborhood heeft (alle similarities zijn $\leq$ 0)? Schrik dus niet als je errors krijgt in functies die je in het vorige deel al goed had bevonden. 

Hieronder staat wat test code voor de functes `select_neighborhood` en `weighted mean`. Hoe goed gaan de functies om met ongeldige input? Zorg ervoor dat deze tests geen fouten geven voor je verder gaat. Mocht dat toch zo zijn, is het aan te raden ze naar de cell hieronder te kopieren en ze daar te debuggen. 

In [ ]:
# TODO


In [ ]:
# Test select_neighborhood and weighted_mean
_test_utility = DataFrame([[1, 1, 1],[-1, -1, -1]], index = ['sample1', 'sample2'], columns = ['feature1', 'feature2', 'feature3'])
_similarity = DataFrame ([[1., -1.], [-1.,  1.]], index = ['sample1', 'sample2'], columns = ['sample1', 'sample2'])

_neigborhood = select_neighborhood(_similarity, _test_utility, 'feature1', 'sample1')
try:
    _neigborhood = select_neighborhood(_similarity, _test_utility, 'feature4', 'sample1')
except:
    raise Exception("Crashed on missing feature: feature4") from None
try:
    _neigborhood = select_neighborhood(_similarity, _test_utility, 'feature1', 'sample3')
except:
    raise Exception("Crashed on missing item: sample3") from None
    
_weighted_mean = weighted_mean(_neigborhood, _test_utility, 'feature1')

try:
    float(_weighted_mean)
except ValueError:    
    raise Exception("expected weighted_mean() is a float") from None
    
_neighborhood2 = Series(0, index=[])
_weighted_mean = weighted_mean(_neighborhood2, _test_utility, 'feature1')

try:
    float(_weighted_mean)
except ValueError:    
    raise Exception("expected weighted_mean() is a float") from None

### Extra helper functies
Hier geven we een aantal extra helper functies die je kan inzetten in de opdracht.

In [ ]:
def number_of_movies(ratings):
    """ determine the number of unique movie id's in the data """
    return len(ratings['movieId'].unique())

def number_of_users(ratings):
    """ determine the number of unique user id's in the data """
    return len(ratings['userId'].unique())

def number_of_ratings(ratings):
    """ count the number of ratings of a dataset """
    return ratings.shape[0]

def rating_density(ratings):
    """ compute the ratings given a dataset """
    return number_of_ratings(ratings) / (number_of_movies(ratings) * number_of_users(ratings))

def split_data(data,d = 0.75):
    """ split data in a training and test set 
       `d` is the fraction of data in the training set"""
    np.random.seed(seed=5)
    mask_test = np.random.rand(data.shape[0]) < d
    return data[mask_test], data[~mask_test]

### Laad de data

Hieronder laden we een nieuwe dataset. Deze bevat ruim 3000 ratings. Om te kunnen testen hoe goed ons algoritme werkt splitsen we de data in twee delen: `ratings_t80_training` (de data waarmee we gaan voorspellen) en `ratings_t80_test` (de data waarmee we testen hoe goed de voorspelling is). 

De dataset heeft een dichtheid van ongeveer 80% (vandaar de uitgang `_t80`). Dat wil zeggen dat gemiddeld elke gebruiker 80% van de films van een rating heeft voorzien. Omdat we een deel van de ratings appart houden voor de test set, blijft daar in de training set maar 60% van over. Dat is trouwens heel veel, in de praktijk haal je vaak niet eens 1%. Wat voor een effect dit percentage heeft op de voorspellingen gaan we in deze opdracht uitzoeken.

In [ ]:
# Load the ratings_t80
ratings_t80 = pd.read_csv('./mini-movielens/ratings_t80.csv')
ratings_t80_training, ratings_t80_test = split_data(ratings_t80)

# Show the stats
print('Stats training set:')
print(f'Number of ratings: {number_of_ratings(ratings_t80_training)}')
print(f'Number of users: {number_of_users(ratings_t80_training)}')
print(f'Number of movies: {number_of_movies(ratings_t80_training)}')
print(f'Density: {rating_density(ratings_t80_training)}')

# Display the first 5 rows of the data
display(ratings_t80.head())

### Vraag 1
Maak een utility matrix in een variabele genaamd `utility` en een _adjusted cosine_ similarity matrix in een variabele genaamd `similarity`. Doe beide voor _item based_ filtering. Gebruik hiervoor de functies die je in deel 1 van de opdracht hebt gemaakt.

Dit kan even duren om te berekenen, misschien zelfs een paar minuten. Het is namelijk aanzienlijk meer data dan in het eerste deel van de opdracht! (De data bevat 200 films, dus de similarity matrix bevat 200 x 200 = 40.000 cellen.)

In [ ]:
# TODO


display(utility.head())
display(similarity.head())

In [ ]:
partial_similarity_solution = pd.read_msgpack(b'\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc8\x06@\x00\x01\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x002\x00\x00\x00\x00\x00\x00\x00h\x00\x00\x00\x00\x00\x00\x00\x99\x00\x00\x00\x00\x00\x00\x00\xdf\x00\x00\x00\x00\x00\x00\x00\xe7\x00\x00\x00\x00\x00\x00\x00\x04\x01\x00\x00\x00\x00\x00\x00%\x01\x00\x00\x00\x00\x00\x00(\x01\x00\x00\x00\x00\x00\x00>\x01\x00\x00\x00\x00\x00\x00d\x01\x00\x00\x00\x00\x00\x00l\x01\x00\x00\x00\x00\x00\x00o\x01\x00\x00\x00\x00\x00\x00y\x01\x00\x00\x00\x00\x00\x00|\x01\x00\x00\x00\x00\x00\x00\xc9\x01\x00\x00\x00\x00\x00\x00\xe0\x01\x00\x00\x00\x00\x00\x00\xf4\x01\x00\x00\x00\x00\x00\x00\x0f\x02\x00\x00\x00\x00\x00\x00\x1d\x02\x00\x00\x00\x00\x00\x00M\x02\x00\x00\x00\x00\x00\x00P\x02\x00\x00\x00\x00\x00\x00`\x02\x00\x00\x00\x00\x00\x00\x88\x02\x00\x00\x00\x00\x00\x00\x0c\x03\x00\x00\x00\x00\x00\x00Z\x03\x00\x00\x00\x00\x00\x00\x97\x03\x00\x00\x00\x00\x00\x00\x9b\x03\x00\x00\x00\x00\x00\x00\x9c\x03\x00\x00\x00\x00\x00\x00\x04\x04\x00\x00\x00\x00\x00\x00\x0c\x04\x00\x00\x00\x00\x00\x00A\x04\x00\x00\x00\x00\x00\x00\xa9\x04\x00\x00\x00\x00\x00\x00\xac\x04\x00\x00\x00\x00\x00\x00\xae\x04\x00\x00\x00\x00\x00\x00\xb1\x04\x00\x00\x00\x00\x00\x00\xb8\x04\x00\x00\x00\x00\x00\x00\xba\x04\x00\x00\x00\x00\x00\x00\xbd\x04\x00\x00\x00\x00\x00\x00\xbe\x04\x00\x00\x00\x00\x00\x00\xc3\x04\x00\x00\x00\x00\x00\x00\xc5\x04\x00\x00\x00\x00\x00\x00\xd8\x04\x00\x00\x00\x00\x00\x00\xea\x04\x00\x00\x00\x00\x00\x00k\x05\x00\x00\x00\x00\x00\x00\xed\x05\x00\x00\x00\x00\x00\x00\xf7\x05\x00\x00\x00\x00\x00\x00\x08\x06\x00\x00\x00\x00\x00\x00,\x06\x00\x00\x00\x00\x00\x00u\x06\x00\x00\x00\x00\x00\x00\x8c\x06\x00\x00\x00\x00\x00\x00\x92\x06\x00\x00\x00\x00\x00\x00\xb9\x06\x00\x00\x00\x00\x00\x00\xc4\x06\x00\x00\x00\x00\x00\x00\x83\x07\x00\x00\x00\x00\x00\x00\xa2\x07\x00\x00\x00\x00\x00\x00\xd5\x07\x00\x00\x00\x00\x00\x00\xdc\x07\x00\x00\x00\x00\x00\x00\xec\x07\x00\x00\x00\x00\x00\x00!\x08\x00\x00\x00\x00\x00\x00C\x08\x00\x00\x00\x00\x00\x00\xf3\x08\x00\x00\x00\x00\x00\x00\x19\t\x00\x00\x00\x00\x00\x00\xa6\t\x00\x00\x00\x00\x00\x00\xc6\t\x00\x00\x00\x00\x00\x009\n\x00\x00\x00\x00\x00\x00D\n\x00\x00\x00\x00\x00\x00\x92\n\x00\x00\x00\x00\x00\x00\x9c\n\x00\x00\x00\x00\x00\x00\x9d\n\x00\x00\x00\x00\x00\x00\xed\n\x00\x00\x00\x00\x00\x00*\x0b\x00\x00\x00\x00\x00\x00f\x0b\x00\x00\x00\x00\x00\x00\x8f\x0b\x00\x00\x00\x00\x00\x00\xa9\x0b\x00\x00\x00\x00\x00\x00\xab\x0b\x00\x00\x00\x00\x00\x00\t\x0c\x00\x00\x00\x00\x00\x00K\x0c\x00\x00\x00\x00\x00\x00\xc7\r\x00\x00\x00\x00\x00\x00\xfa\r\x00\x00\x00\x00\x00\x00\'\x0e\x00\x00\x00\x00\x00\x00\x89\x0f\x00\x00\x00\x00\x00\x00\x9a\x0f\x00\x00\x00\x00\x00\x00\xb6\x0f\x00\x00\x00\x00\x00\x00\xbb\x0f\x00\x00\x00\x00\x00\x00\x82\x10\x00\x00\x00\x00\x00\x00\xa6\x10\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00#\x12\x00\x00\x00\x00\x00\x00p\x12\x00\x00\x00\x00\x00\x00\x0e\x13\x00\x00\x00\x00\x00\x00\x16\x13\x00\x00\x00\x00\x00\x00\x1a\x13\x00\x00\x00\x00\x00\x00 \x13\x00\x00\x00\x00\x00\x00\x81\x13\x00\x00\x00\x00\x00\x00\x83\x13\x00\x00\x00\x00\x00\x00\x02\x15\x00\x00\x00\x00\x00\x00E\x15\x00\x00\x00\x00\x00\x00~\x15\x00\x00\x00\x00\x00\x00\xf2\x15\x00\x00\x00\x00\x00\x00\xb8\x16\x00\x00\x00\x00\x00\x00e\x17\x00\x00\x00\x00\x00\x00\xbd\x18\x00\x00\x00\x00\x00\x00\xdd\x18\x00\x00\x00\x00\x00\x00\xe5\x18\x00\x00\x00\x00\x00\x00\xe9\x18\x00\x00\x00\x00\x00\x00\xea\x18\x00\x00\x00\x00\x00\x00\x8b\x19\x00\x00\x00\x00\x00\x007\x1a\x00\x00\x00\x00\x00\x00\xda\x1a\x00\x00\x00\x00\x00\x00\x16\x1b\x00\x00\x00\x00\x00\x00\xf1\x1b\x00\x00\x00\x00\x00\x00\xc1\x1c\x00\x00\x00\x00\x00\x00\xb0 \x00\x00\x00\x00\x00\x00\xbc!\x00\x00\x00\x00\x00\x00\xd9!\x00\x00\x00\x00\x00\x00\xaa"\x00\x00\x00\x00\x00\x00\xd5\x82\x00\x00\x00\x00\x00\x00\x8f\x83\x00\x00\x00\x00\x00\x00\x02\x84\x00\x00\x00\x00\x00\x00\x00\x85\x00\x00\x00\x00\x00\x00\xfc\x8b\x00\x00\x00\x00\x00\x00o\x9f\x00\x00\x00\x00\x00\x00^\xa2\x00\x00\x00\x00\x00\x00\x9f\xac\x00\x00\x00\x00\x00\x00\xbb\xb1\x00\x00\x00\x00\x00\x00\x9a\xb2\x00\x00\x00\x00\x00\x00\xf2\xb5\x00\x00\x00\x00\x00\x00\x01\xbd\x00\x00\x00\x00\x00\x00\n\xbd\x00\x00\x00\x00\x00\x00\x84\xbd\x00\x00\x00\x00\x00\x00\xce\xc9\x00\x00\x00\x00\x00\x00\xf2\xcd\x00\x00\x00\x00\x00\x00\x85\xcf\x00\x00\x00\x00\x00\x00\xbf\xe4\x00\x00\x00\x00\x00\x00\xb3\xe7\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00/\x00\x00\x00\x00\x00\x00\x00n\x00\x00\x00\x00\x00\x00\x00Q\x02\x00\x00\x00\x00\x00\x00\n\x03\x00\x00\x00\x00\x00\x00\xc6\x04\x00\x00\x00\x00\x00\x00\xa8\x06\x00\x00\x00\x00\x00\x00\xca\n\x00\x00\x00\x00\x00\x00\xc9\x0e\x00\x00\x00\x00\x00\x00\xd1\x0e\x00\x00\x00\x00\x00\x00l\x0f\x00\x00\x00\x00\x00\x00\x9c\x0f\x00\x00\x00\x00\x00\x00\xab\x0f\x00\x00\x00\x00\x00\x00c\x13\x00\x00\x00\x00\x00\x00b\x14\x00\x00\x00\x00\x00\x00*\x15\x00\x00\x00\x00\x00\x00S\x15\x00\x00\x00\x00\x00\x00/\x16\x00\x00\x00\x00\x00\x00D\x17\x00\x00\x00\x00\x00\x00\x80\x17\x00\x00\x00\x00\x00\x00\xcf\x1a\x00\x00\x00\x00\x00\x00\x0e\x1d\x00\x00\x00\x00\x00\x00\xa8 \x00\x00\x00\x00\x00\x00\x01#\x00\x00\x00\x00\x00\x00Ix\x00\x00\x00\x00\x00\x00K\x7f\x00\x00\x00\x00\x00\x00\x8c\xbe\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00X\x01\x00\x00\x00\x00\x00\x00\xee\x02\x00\x00\x00\x00\x00\x001\x04\x00\x00\x00\x00\x00\x00I\x04\x00\x00\x00\x00\x00\x00\xad\x04\x00\x00\x00\x00\x00\x00\xb6\x04\x00\x00\x00\x00\x00\x00\xc4\x04\x00\x00\x00\x00\x00\x00\xf1\x04\x00\x00\x00\x00\x00\x00\xf6\x04\x00\x00\x00\x00\x00\x00\x0b\x05\x00\x00\x00\x00\x00\x00d\x05\x00\x00\x00\x00\x00\x00\x10\x06\x00\x00\x00\x00\x00\x00\xb0\x07\x00\x00\x00\x00\x00\x00\xdb\x07\x00\x00\x00\x00\x00\x00\x0b\n\x00\x00\x00\x00\x00\x00\xec\x0b\x00\x00\x00\x00\x00\x009\x0f\x00\x00\x00\x00\x00\x00\xdb\x11\x00\x00\x00\x00\x00\x00\x90\x03\x00\x00\x00\x00\x00\x00M\x04\x00\x00\x00\x00\x00\x00p\x04\x00\x00\x00\x00\x00\x00{\n\x00\x00\x00\x00\x00\x00\xb5\x0b\x00\x00\x00\x00\x00\x00*\x0c\x00\x00\x00\x00\x00\x00\xa8\x12\x00\x00\x00\x00\x00\x00\xe5\x14\x00\x00\x00\x00\x00\x00@\x17\x00\x00\x00\x00\x00\x00S\x02\x00\x00\x00\x00\x00\x00\xe7\n\x00\x00\x00\x00\x00\x00\x12\x11\x00\x00\x00\x00\x00\x00\xb0\x04\x00\x00\x00\x00\x00\x00\xd0\x07\x00\x00\x00\x00\x00\x003\t\x00\x00\x00\x00\x00\x00~\x08\x00\x00\x00\x00\x00\x00\xdf\x04\x00\x00\x00\x00\x00\x00\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7(\x00\x01\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x002\x00\x00\x00\x00\x00\x00\x00h\x00\x00\x00\x00\x00\x00\x00\x99\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x91\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\xcc\xc8\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc8\x06@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\x06\x00\x00\x00\x00\x00\x00\x00\x07\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\t\x00\x00\x00\x00\x00\x00\x00\n\x00\x00\x00\x00\x00\x00\x00\x0b\x00\x00\x00\x00\x00\x00\x00\x0c\x00\x00\x00\x00\x00\x00\x00\r\x00\x00\x00\x00\x00\x00\x00\x0e\x00\x00\x00\x00\x00\x00\x00\x0f\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x00\x11\x00\x00\x00\x00\x00\x00\x00\x12\x00\x00\x00\x00\x00\x00\x00\x13\x00\x00\x00\x00\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00\x15\x00\x00\x00\x00\x00\x00\x00\x16\x00\x00\x00\x00\x00\x00\x00\x17\x00\x00\x00\x00\x00\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00\x19\x00\x00\x00\x00\x00\x00\x00\x1a\x00\x00\x00\x00\x00\x00\x00\x1b\x00\x00\x00\x00\x00\x00\x00\x1c\x00\x00\x00\x00\x00\x00\x00\x1d\x00\x00\x00\x00\x00\x00\x00\x1e\x00\x00\x00\x00\x00\x00\x00\x1f\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00!\x00\x00\x00\x00\x00\x00\x00"\x00\x00\x00\x00\x00\x00\x00#\x00\x00\x00\x00\x00\x00\x00$\x00\x00\x00\x00\x00\x00\x00%\x00\x00\x00\x00\x00\x00\x00&\x00\x00\x00\x00\x00\x00\x00\'\x00\x00\x00\x00\x00\x00\x00(\x00\x00\x00\x00\x00\x00\x00)\x00\x00\x00\x00\x00\x00\x00*\x00\x00\x00\x00\x00\x00\x00+\x00\x00\x00\x00\x00\x00\x00,\x00\x00\x00\x00\x00\x00\x00-\x00\x00\x00\x00\x00\x00\x00.\x00\x00\x00\x00\x00\x00\x00/\x00\x00\x00\x00\x00\x00\x000\x00\x00\x00\x00\x00\x00\x001\x00\x00\x00\x00\x00\x00\x002\x00\x00\x00\x00\x00\x00\x003\x00\x00\x00\x00\x00\x00\x004\x00\x00\x00\x00\x00\x00\x005\x00\x00\x00\x00\x00\x00\x006\x00\x00\x00\x00\x00\x00\x007\x00\x00\x00\x00\x00\x00\x008\x00\x00\x00\x00\x00\x00\x009\x00\x00\x00\x00\x00\x00\x00:\x00\x00\x00\x00\x00\x00\x00;\x00\x00\x00\x00\x00\x00\x00<\x00\x00\x00\x00\x00\x00\x00=\x00\x00\x00\x00\x00\x00\x00>\x00\x00\x00\x00\x00\x00\x00?\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00A\x00\x00\x00\x00\x00\x00\x00B\x00\x00\x00\x00\x00\x00\x00C\x00\x00\x00\x00\x00\x00\x00D\x00\x00\x00\x00\x00\x00\x00E\x00\x00\x00\x00\x00\x00\x00F\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00H\x00\x00\x00\x00\x00\x00\x00I\x00\x00\x00\x00\x00\x00\x00J\x00\x00\x00\x00\x00\x00\x00K\x00\x00\x00\x00\x00\x00\x00L\x00\x00\x00\x00\x00\x00\x00M\x00\x00\x00\x00\x00\x00\x00N\x00\x00\x00\x00\x00\x00\x00O\x00\x00\x00\x00\x00\x00\x00P\x00\x00\x00\x00\x00\x00\x00Q\x00\x00\x00\x00\x00\x00\x00R\x00\x00\x00\x00\x00\x00\x00S\x00\x00\x00\x00\x00\x00\x00T\x00\x00\x00\x00\x00\x00\x00U\x00\x00\x00\x00\x00\x00\x00V\x00\x00\x00\x00\x00\x00\x00W\x00\x00\x00\x00\x00\x00\x00X\x00\x00\x00\x00\x00\x00\x00Y\x00\x00\x00\x00\x00\x00\x00Z\x00\x00\x00\x00\x00\x00\x00[\x00\x00\x00\x00\x00\x00\x00\\\x00\x00\x00\x00\x00\x00\x00]\x00\x00\x00\x00\x00\x00\x00^\x00\x00\x00\x00\x00\x00\x00_\x00\x00\x00\x00\x00\x00\x00`\x00\x00\x00\x00\x00\x00\x00a\x00\x00\x00\x00\x00\x00\x00b\x00\x00\x00\x00\x00\x00\x00c\x00\x00\x00\x00\x00\x00\x00d\x00\x00\x00\x00\x00\x00\x00e\x00\x00\x00\x00\x00\x00\x00f\x00\x00\x00\x00\x00\x00\x00g\x00\x00\x00\x00\x00\x00\x00h\x00\x00\x00\x00\x00\x00\x00i\x00\x00\x00\x00\x00\x00\x00j\x00\x00\x00\x00\x00\x00\x00k\x00\x00\x00\x00\x00\x00\x00l\x00\x00\x00\x00\x00\x00\x00m\x00\x00\x00\x00\x00\x00\x00n\x00\x00\x00\x00\x00\x00\x00o\x00\x00\x00\x00\x00\x00\x00p\x00\x00\x00\x00\x00\x00\x00q\x00\x00\x00\x00\x00\x00\x00r\x00\x00\x00\x00\x00\x00\x00s\x00\x00\x00\x00\x00\x00\x00t\x00\x00\x00\x00\x00\x00\x00u\x00\x00\x00\x00\x00\x00\x00v\x00\x00\x00\x00\x00\x00\x00w\x00\x00\x00\x00\x00\x00\x00x\x00\x00\x00\x00\x00\x00\x00y\x00\x00\x00\x00\x00\x00\x00z\x00\x00\x00\x00\x00\x00\x00{\x00\x00\x00\x00\x00\x00\x00|\x00\x00\x00\x00\x00\x00\x00}\x00\x00\x00\x00\x00\x00\x00~\x00\x00\x00\x00\x00\x00\x00\x7f\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x81\x00\x00\x00\x00\x00\x00\x00\x82\x00\x00\x00\x00\x00\x00\x00\x83\x00\x00\x00\x00\x00\x00\x00\x84\x00\x00\x00\x00\x00\x00\x00\x85\x00\x00\x00\x00\x00\x00\x00\x86\x00\x00\x00\x00\x00\x00\x00\x87\x00\x00\x00\x00\x00\x00\x00\x88\x00\x00\x00\x00\x00\x00\x00\x89\x00\x00\x00\x00\x00\x00\x00\x8a\x00\x00\x00\x00\x00\x00\x00\x8b\x00\x00\x00\x00\x00\x00\x00\x8c\x00\x00\x00\x00\x00\x00\x00\x8d\x00\x00\x00\x00\x00\x00\x00\x8e\x00\x00\x00\x00\x00\x00\x00\x8f\x00\x00\x00\x00\x00\x00\x00\x90\x00\x00\x00\x00\x00\x00\x00\x91\x00\x00\x00\x00\x00\x00\x00\x92\x00\x00\x00\x00\x00\x00\x00\x93\x00\x00\x00\x00\x00\x00\x00\x94\x00\x00\x00\x00\x00\x00\x00\x95\x00\x00\x00\x00\x00\x00\x00\x96\x00\x00\x00\x00\x00\x00\x00\x97\x00\x00\x00\x00\x00\x00\x00\x98\x00\x00\x00\x00\x00\x00\x00\x99\x00\x00\x00\x00\x00\x00\x00\x9a\x00\x00\x00\x00\x00\x00\x00\x9b\x00\x00\x00\x00\x00\x00\x00\x9c\x00\x00\x00\x00\x00\x00\x00\x9d\x00\x00\x00\x00\x00\x00\x00\x9e\x00\x00\x00\x00\x00\x00\x00\x9f\x00\x00\x00\x00\x00\x00\x00\xa0\x00\x00\x00\x00\x00\x00\x00\xa1\x00\x00\x00\x00\x00\x00\x00\xa2\x00\x00\x00\x00\x00\x00\x00\xa3\x00\x00\x00\x00\x00\x00\x00\xa4\x00\x00\x00\x00\x00\x00\x00\xa5\x00\x00\x00\x00\x00\x00\x00\xa6\x00\x00\x00\x00\x00\x00\x00\xa7\x00\x00\x00\x00\x00\x00\x00\xa8\x00\x00\x00\x00\x00\x00\x00\xa9\x00\x00\x00\x00\x00\x00\x00\xaa\x00\x00\x00\x00\x00\x00\x00\xab\x00\x00\x00\x00\x00\x00\x00\xac\x00\x00\x00\x00\x00\x00\x00\xad\x00\x00\x00\x00\x00\x00\x00\xae\x00\x00\x00\x00\x00\x00\x00\xaf\x00\x00\x00\x00\x00\x00\x00\xb0\x00\x00\x00\x00\x00\x00\x00\xb1\x00\x00\x00\x00\x00\x00\x00\xb2\x00\x00\x00\x00\x00\x00\x00\xb3\x00\x00\x00\x00\x00\x00\x00\xb4\x00\x00\x00\x00\x00\x00\x00\xb5\x00\x00\x00\x00\x00\x00\x00\xb6\x00\x00\x00\x00\x00\x00\x00\xb7\x00\x00\x00\x00\x00\x00\x00\xb8\x00\x00\x00\x00\x00\x00\x00\xb9\x00\x00\x00\x00\x00\x00\x00\xba\x00\x00\x00\x00\x00\x00\x00\xbb\x00\x00\x00\x00\x00\x00\x00\xbc\x00\x00\x00\x00\x00\x00\x00\xbd\x00\x00\x00\x00\x00\x00\x00\xbe\x00\x00\x00\x00\x00\x00\x00\xbf\x00\x00\x00\x00\x00\x00\x00\xc0\x00\x00\x00\x00\x00\x00\x00\xc1\x00\x00\x00\x00\x00\x00\x00\xc2\x00\x00\x00\x00\x00\x00\x00\xc3\x00\x00\x00\x00\x00\x00\x00\xc4\x00\x00\x00\x00\x00\x00\x00\xc5\x00\x00\x00\x00\x00\x00\x00\xc6\x00\x00\x00\x00\x00\x00\x00\xc7\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc8\x1f@\x00\x00\x00\x00\x00\x00\x00\xf0?\xb9\xd0\xd3\xb0\xb1\xc5x?\xc9G{\x0f\x9d\xac\xc6?\xa9V\x01\x87\r\x90\xe3\xbf6\xb1\x83\xe8\xdf\x9b\xd7\xbf\xb9\xd0\xd3\xb0\xb1\xc5x?\x00\x00\x00\x00\x00\x00\xf0?\xafw\x06q+\xac\xe6?>\xfd\xa7\x00o\xec\xc1\xbfo\r\x19/Z\xd6\xc2?\xc9G{\x0f\x9d\xac\xc6?\xafw\x06q+\xac\xe6?\x00\x00\x00\x00\x00\x00\xf0?\x94\xf8\xb2\xc1\x13\xd8\xe0\xbfP\x1a\xf0\xa2\x97\xb3\xd8\xbf\xa9V\x01\x87\r\x90\xe3\xbf>\xfd\xa7\x00o\xec\xc1\xbf\x94\xf8\xb2\xc1\x13\xd8\xe0\xbf\x00\x00\x00\x00\x00\x00\xf0?u/\xa5]\xe4\x07\xe0?6\xb1\x83\xe8\xdf\x9b\xd7\xbfo\r\x19/Z\xd6\xc2?P\x1a\xf0\xa2\x97\xb3\xd8\xbfu/\xa5]\xe4\x07\xe0?\x00\x00\x00\x00\x00\x00\xf0?\xf9\xfc\xb6 Z\x10\xc6? \xd4{\xee\x98\xe3\xb4\xbf\xe5E\xe1M\x1e\x9d\xd4?g\x7f\xe5\xcf\xb7\x91\xc3\xbfO\x96\x85\xd1\xd4{\xbe?-\x96\x84\xa7\xbf>\xc0\xbfc\xd7\x13\x83\xde\x15\xe2\xbf\x1eM^v\xf4\xae\xe5\xbf"\xf07\xe1O\xc9\xd9?\x8dj\xc5\xa9\x1c\xda\xe7?qH\xbe$\xb6x\xc0\xbf\xbd\xeb\x9c\x990\xca\xd5\xbfl\xaa\xa1=c\x11\xd0?5Q\xca\x7f\xad\x13\xb2\xbf\xd0(A4\x12\xa8\xe2\xbfX\x83+G\xe6\xee\xdb\xbf^\x1e<\x90\xc9\xcf\xd0?y"\rg=\xf9\xdb?2\xe7X\xdf6\xe1\xd6\xbf\xe08p+\xf0|s?\x96w\x87\x05\xfdE\xd6?\xa0O\xb3\xfc\xbc\xed\xd9? \'\xf1\xae\x102\xea?\xef\xdd4\x99\xde\xc4\xe3\xbfC@\x9a"\xbc\x88\xe5\xbf\x82\xd3\xdamY\xdb\xd0?E:)TI\xb9\xd8?t\xae&@\xcd8\xe4?\xa6N\x8d\xbea\x96\xd1\xbf\xaa\xbb\xb4:^\xd7\xe0\xbf\xabg=\t\x05\x12\xb1?\x93F\xa6\xad\x81\x93\xd7\xbf\xc9?\xb6\xde\xc7\xee\xd7?&\x8b\xf1\xd2\xc8\x1b\xd0\xbf\xd82\xf9ep\xcd\xde\xbf\x98\xa4\x08]\x8cW\xb2?\xf4\xbc1\x1f\x9a\xb1\xcc\xbf\xb5\'ls\xbf\x91\xb2\xbf\x11\xc4O\xd9\xc1\xf4\xb0\xbf%\x05R\xba\xce\xbb\xc6\xbf\x1dW\xd2\x9aP\xf4\xc9\xbf\x9a\xf1\xb3\x92\x95t\xe3\xbf1%\xbb\xa9\x1c\x0c\xe4\xbf\x96\x90Q\x1f\xd5\xd7\xe3?\xa3\xc1\xd6"e5\xe5?Q\xe3\xec\xe48\xfe\xd3\xbf\x8e\xd6\x9e\xccHZ\xbf\xbf\x9f;\xfc\x99lx\xeb\xbf\x14\x16\xd2\xe7\x10\xc9\xd8?"\x88\xab+\xf4\x16\xa9?o\xd4\xcd\xf3\xe80\xd0\xbf\x00\xad\xee\xd4\xa7z\xd6\xbf\x89\x92|_2\xce\xdf\xbfS3>y)D\xe2?)\x15ng\xe2\xc5\xe6?)\x95R\x03\r\xaf\xc5\xbf|\x89\x152\xdc\xce\xd3\xbfCP\xab\xe9\xd6\xf2\xd7\xbfv\xfa\xea\xf8$\xa1\xc2\xbf~g\xfb~\xc3\xc2\xd5\xbf\x15R9ac\xbe\xa5\xbf\xe6\xf8\x17\xde\x13\xef\xd5\xbf\xc6B\xb0\xb1u\x0f\xcc\xbf\xb3\x8d=m\x1a\x91\xd3\xbf\x02\x90A\xd7\x90\xbc\xd2\xbfs\x86\xe7\xfa\xfeN\xc8\xbf\xc1\x06?9\xab\xc5~?\xee\x85Kh\xf6t\xe3\xbf&\xd27\x16\xa6\x9f\xd3?\\(7B\x11\x94\xe6?7\xc6\xf2\x11_\xcb\xc9?5z\x1f[\x0e\x97\xd0?!.\xe7\xa2:\xfd\xe5?\xc8\xa6\x97\xe6\xc2o\xe1\xbf\xe3\xdf\xa2&OU\xe3\xbfz\xa9<F\xe4(\xde?Q\x1f\xb9\xa1\xb3\xe9\xe2\xbf\x05\xad\xa9\xde\x1b\xfe\xd6?(\x93~AR\x12\xd3\xbf$q\x03\x82L-\xd0\xbf\xa0P\x0c\x05\xc1\x16\xdc\xbf}\xb4\xab\xf6m\x04\x8e\xbf\t\x185\x95\xc7\x94\xd3\xbf\xd35\xad\x12\xe2\x84\xbf?\x120a#p\r\xd4\xbf\xa0^\xe7]]K\xd9\xbf\xb3\xf5\xc0\xc5\xedi\xa6?MK\xa8\xadZ\xa1\xae\xbfQ]\x97\x11EO\xe0?\xa81\x98\x15\x8fA\xd2?\x16q\x95\xed\x06\x1d\xd8?\xda\x81@C\xa9\xbf\xc4?S\x19{\x88\xa0\xc4\xe4?b*S#W\xe8\xe4\xbf7t\xb9\xea\xe1\xb1\xe8\xbf\x86\xd8\xeaI\xd4\xb0\xc9\xbf\xa4\xd6\x83\xeeX\x82\x9d\xbf\x0e\x8bJ\xbd\xf6\xf4\xdf\xbf\xf7K\x82\xebp\xff\xde?T\xc8(\xa6$\xaa\xe7?3\xc8+\xc3\xac\xd6\xb2\xbf\xb8*\xcb\xa6\xad\xb2\xd4\xbfO\x8e\x84\xa2\x89\x07\xe5\xbfe\xe3sZu\xde\xdf?\x94\x02\x01\xbbq\x97\xe3?-\xaf\xf8\xae\xf2i\xb4\xbf\xed\xd57\xfc\xdf\xe6\xc9?\xfd\xcc\\\xe6\x93\xdf\xe1?\x04\x89\xf8\xab\x169\xe1\xbf\xa6\x93\x87\rBr\xe9\xbfo\xa8\x95\xe2\x16J\xc9?N;\xb5\xb4\xda\x8c\xcb?5\x84\xef\xba5\xc4\xa3\xbfK\x88\x84\xe6dk\x83\xbf\x85\xa1\xefv~S\xd1\xbfR\xaaM\x81\x1a\x11\xd6?j \x04\xf2\x17\xde\xcd\xbf4\x18\x998\xfe\x0e\xc7?n\xa5xH\x1a\x1e\xde\xbf\xcc\xe4\x10QE1\xe4\xbf\xf9\x05\x87\xd2\xed\xf3\xa7?\x99\x1a\x9fi{nq?\xba\xf3yW\xa0T\xe3?\x0coj\xec\x15\x8d\xda\xbf^\xea\xf7!\xbc\x06\xdd\xbf\xe8\xd9\xcd\xa5qu\xe5?\xcdbm\xe3}\xc8\xcf?\xb7>\x96\xaa\x9eB\xcf\xbf\x1bVs\xf3\xc9\x88\xac?\x85\xa1m\xebkX\xac\xbf\xf7Nc\x0e&\xd5\xda?~\xb3g\xb7\xc5~\xcc\xbf,+z\xc6\x19\xf2\xc2\xbf\xebX\x89\xe1\x9fhs?p\n\xc7\xbb\xbb\xbf\xa9\xbf\n\xcb\xd4\x12#\xa6\xda?\xc1\xdd\xcfTh\xf8\xb2\xbf\xb7\xe6\xb2\xa8\x87T\xe0?\xc7\xe4\x12\xccj\x8a\xe2\xbf\x8f\xc8\x8c\xe2\xf7\xa7\xe6\xbf\xfc\xcatL\xc8\x80\xd1?N\xf5\xf8\t\xb3;\xc2?\xab\xa1\xecP+\xfe\xdb?\x08\x89\x8c\x93\x01"\xc8\xbf\xc2\xdf\x9f\x8dQ\xb2\xd3\xbf\xa6\xd7}\x8c\x06\xb6\xc2\xbf\xe7\xb6e\xe1-W\xae\xbf\xe4\t]\xf9\xa8\xf5\xbe?\xffs\x19\x06\x01\x06\xd4\xbfM5\x11\xfc\x85d\xdf\xbf\x05tU\xab\x87\xaf\xd5?O3^\xd3=D\xc5\xbf\xd8\xcf\xa9\xbf\x05o\xc7? \xb6\x08E\x19n\xb7\xbf\x08\xc5\xcd\tV\xb8\xe9\xbfhB\x8b\xe9\xcdx\xc6?\x84\x01\xa6b\x13\x8a\xe1\xbf\xf2\x19j\x18\x17\xca\xe0?/\x954\x9fA\x15\xd2\xbfR<\x91\xc8gV\xed\xbf\xff\x0cI2\x1a\x1b\xc0?`\x14\xb7\xc9\x96\xf8\xd7?\xdaX\x80\x8e~T\xe8?\x99\xce\xcaAn\x8d\xdd\xbf\xd6\xea\xf3n\xf7\x94\xe5\xbf\xbeA\x8d\x83V\xcf\xc6?\xe7 K\xcc\xcf\x86\xd8\xbf+B\xb5x\x0c\xcf\xd0?[\xe6\xc0\x91\xff\xff\xbc\xbf\xf3o\xc5\x0f\\\xcf\xe0\xbf6\xf5\xb2J7\xe8\xda?\xc1\x0c*|\x17\xe7\xda?\xccG\xe7\xe6\xbe5\xe6?\xb7\xdc\xfc\xa4\xf5/\xcb\xbf\xf6\xebv\xe6};\xe8\xbf\x12+h\xae\x8f\x9a\xc0\xbfu\xd9W%\x96\x05\xcc\xbf\x18\x80O\x0f\xcc\xe0\xb4?\xb9\xa8\x01,\x08\xd2\xd1\xbf\xe4}$\xc7\xe2\xd5\xdc\xbf\xb4x&Y\xfd\x87\xa5?\xcd\xc7`\xfeA|\xc2\xbf\xb6n\xd5\xab`\xac\xa2?y>@,\x8d7\xbc\xbf\x16\xc5\x9e\xde$\xb2\xe7\xbf\xf5\x0b\xe2\xef\x19\xb0\xdd?Sj\xc4i\xac\x9f\xa5?\xc9\xff\xe3\x17\xff\xed\xe5?\x95{)n\x05I\xea\xbf\x87,\x00}X$\xec\xbfeF3t\xbf\xc7\xdc\xbf|\xa4,\x00T\x97\xe6\xbfdC\x90\x9b\x14<\xd0\xbfd\xfc\xd6/"R\xb8?*\x05\x0f\xfdc;\xdf\xbf\xc5\x80/\xfb\xd3q\xd0\xbf\xb2\xa6\x1b\xc6|\xb1\xe0?L\x8c\x16\xa1\xd8)\xe1?\xd7\x0e\x8e\xa4:\xf8\xe2\xbf\xad\xb7\xb8x\xa8\x89\xd3\xbf\xed\xb0\xef`\x1d\x05\xd3?\xee,\x7fN\x15\xb0\xd6\xbfg\xce{\xd2)\xba\xd5\xbfH=\xd9y_\x08\xd8?@u\xb4\x03\xb1(\xc0?M\xf9\x8f\x82*\xb7\xc2\xbf!\x9d\xaa\xda\xf7_\xca\xbf\xf2\x81<w\x95\xc1\xdd\xbf\xa6X\xa6/\x1d8\xde?\xe5\xc1\xd3\xcep\xc9\xdd?Gq\xeb\xff\xf0\x90\xa9\xbf~xc\xfa\x95\xf4\xb5?\x9cv\xbde@\xea\xc6?\xb1\x95\xe4)\x0c\xa6\xc5?\x83\xccE\x9d\xea\x92\xc6\xbf\xf0\xcaL\'\xdf\x12\xcd\xbf\x02Q\xd9V\xd8\xda\xc8?|B\x17u\xf5@\xb9\xbf0\xbf\x12\x8b\xa06\xe4?K\x15\x99\xd6~R\xeb?\xce+\x9e\x0bd8\xc1?\xb0\xa8\xeeD\x95/\xe4\xbf\x90=\xe4T-\xb8\xe7\xbfY\x931\xea_\xe5\xbf\xbf\xd47\xa0\xebS\x86\xb0?\xb92\xb0\xcb\xf5\x86\xe1?\x10\xdaT\x15\xca*\xc4?\xf8\xa7\xb0ee\x9a\xb5\xbfj\x96\xba\xac:y\xda\xbf1L9\xfa\xfb\xdd\xc8\xbf\x95b+\xaa\xc1>\xe4?\rQ\x8a{\x9e\x82\xc3?m\xca`7\xf9i\xa9?\x1eB3\xe8\xb7[\xcc\xbfo\x84\xe7\x83\xbb\xab\xa0?\xf82\x95\xc1\x9dt\xd7\xbfb\x12\xd0JV3\xb6?J-\xd7l\xa6\x84\xb0?\x1f\xaet}3\xd4\x97?9\xdc\x84\x07\x9d\x90\xa4\xbf\x8e\xd5\x1c\xb0\xaa\t\xb3?\x88\xca\x85\xa2qx\xd5\xbfO\x08=r[\x8a\xc1\xbfu\xcf\x8d\x18\x1d\x15\xc6?\xed98\x0fM\xa6\xdb?\x1a\x11\x1fU\xaf\xac\xb1?\x85R\xd7\xe1\xeb\xe1\xb6?\xf73|vrO\xe0?\x8b\xd6l\xf3\xd30\xc6\xbf\x89\x13\xf9\xe8\x13\x11\xd6\xbf\xef\xde\ti-\x9f\xc4?r\x83\x8e$\xe8|\xe0\xbfy\xb4\x00\xe8\x04R\xeb\xbfQ\x97\xa2\x1d#\xfa\xdd?\xa9\xefq\xbc\x1a|\xc8\xbf\x0e\xdeF\xc55\x18\xe6?\x8d\x81rH}\x18\xe0\xbf\xe0\x94\xd8\x97\xb0\xc4\xe0\xbf\x8aw\xbe\xa4|$\xdd\xbf!\xa9\xbe\xf6gM\xe4\xbfL\\\xe3y\xb1b\xb9\xbfn\x97\xd4\x01u\'\xb1?\xf47\xe4A\x01\xbd\xa2\xbf\xf6\xbd8\xb9o\x82\xd1?3U\xda\xdc\x1fB\xd4\xbflG8\x82\xb75\xd0?\xe9|IyW\xc0\xc1\xbfn\x08|`P\x8d\xdb\xbf\\`\xfa\x0b\xb7\xe7\xd2?\xf0,\x9cn\xe6\xe5\xe2?Et\t|\xf5S\xb0?\xe3A\x16\xce\xd76\xbf\xbf\x91\x83Y\x920\x14\xb7?\x18\x96-\xd0\x93+\xd9\xbf\xa2\x1f\xe2\xe9\x97\xdc\xeb\xbf]`H\xf8\xf0\xf2\xd3\xbf\xc7`\xa7\x10\xdb%\xd8\xbf\xe3\x98P\xbc\xc0D\xe1\xbfs\xfb<&\xb2D\xe1?P\xfa\xb2\x95/\xf7\xd9?\x91\xf4 \xaeW\x8f\xcf\xbf\xc2\xcf\xe2M\xec\xb9\xaf?\xd4!\xb2\x12\x16X\xa9\xbf\x1b(\x82\xe2,)\xe5?\xa2h\x19[\x8b\xd4\xda?\xa2m\xe4\xf0\xb2\xaa\xb3\xbfc\xbe\t\xf9\xf9\x18\xa9?\x97I1\xec\xae\xf4\xe4?\x90\x12\xe9\xb9\x0b\x9a\xe4\xbf#\xd3i+]v\xe0\xbf5]n\xe4O\xe4\xe2?\x8dt\xe9\x85\xff\xb8\xc5?Z\xd3\xfc\xfd\xa3\xc4\xe1?\x1b\xf4\xa6AA\xcb\xe1\xbfGmuwE\x9e\xdf\xbf\xd3\xf1\x14\x90\x00\xf3\xd1\xbf<\xaa\xfa\x0c\xbc\xbe\xe2\xbf\x0b\xc7\n_\x84S\xe6\xbf\x8b\xf1t\xbaU\xb4\xe0?\xef\x149\xbf\xedz\xe8?\xfa;k\x03\xd9\x1c\xe0\xbfl\xbb\x9bH\xb2\xda\xdd?\x9b\xbfz\x9dbu\xc3?\x9c\xed\xc2\x89\x91\xcb\xb5\xbf\x9f\xce\xbc^\xd9\xa3\xdc\xbfS3\x8f\xc5E\xb8\xdd?\xff\xc3\xf2\xbe\x99%\xd9?\x10\xc8\x9c?\x04\x92\xea\xbfx\xceu\xe2\x9e]\x82\xbf\xe5j\x19\x18\x18\x9f\xe3?8\xdb \x12\x0b\xa3\xd0?\x13\xf3\x1ee\xef\x96\xdc\xbf\xd7\xe9\xa1P\x85\xac\xc4\xbf\x15\x19T\xbf\xb2\x8c\xd5?2t\xd7\xca\x0b\x10\xdf?Y\n\x84\xa4\xbc\x83\xdf\xbf\x10\xda\xdc\xcfip\xe9\xbf\xc5\xd9\x03.\xc6T\xeb\xbfA\xc9\r"\xaeM\xe7?\xe9"\xc9\xcb&\xd9\xe0?\x1e\xb8\x8d#\xfd\x14\xdb?\x98\\\xf7\xc6c\xd9\xe2\xbfV\x90[\x0c\xb3\xc1\xc0?h\xfd=\x9d\x16\xbd\xe3\xbf\xd3\x194\xa1\x87\xc9\x81\xbf\x0f\x96\x1a:\x07\xb9\xd6\xbf\x1a\xca\xe8\x0b\x8a\xa6\xb1?M\x9a\xd6Tr\xbe\xd6\xbf\xf6\x16\x85\x9a\x83\t\xe3?)\x14\xdf\xa8/p\xed?\x8a\xb5SN?A\xcc?\xbf}\t\xe8\xfd\xbe\xca?\x96\xf3\x90\xa8\xc7\xa6\xd5\xbfQs\xd3>\x8c\xf7\xd8?6]\x0e"\x11B\xa1\xbfm\x1d;\x9fQ\xed\xd3\xbf\xb2Q;\xce\xec\xc8\xe1?.\x9e\x16\x04D\xc5\xde?=\xb7\xcf\xd9\x14\xab\xcd\xbfu^\x01\xc0\xa1\x8d\xe0\xbfu\x7fcs\xcfS|\xbf\x93\x95\x7f1ab\xca?s\xfbc\xc6\xfe\xee\xd2?\xd6~\xa5\xae\x91j\xb3\xbf\xf3~K\x99`9\xd1\xbf\x92<>k\xc6\x9b\xc7\xbf\xdd67q\x12W\xdf?\x9a\xf6\x1b\xf8\x93\x05\xe7?\x97+\xa4\x04r\xd8\xd9\xbf\x95\xca\x8co\t\x11\xe1\xbfc\xa3\xb4\xe4\xcf\x1b\xd4?\xc27(\xb1\xdd\x7f\xe2\xbf\xfb\x99G\xb5\x87\xee\xe3\xbf\x96WL.\xffd\xd2?\xe7B\xdc\xab\xe6\xbd\xdf?/\xa9:\x020\x85\xb9?_\xa0\xde\xcf\xebV\x81\xbf\xcb\xf6\x18\xf3\x96\x0b\xa7\xbf\xc15q\xb7\xa0\x10\xd4\xbf\x8f\xa8\'\xaaX\x84\xe0?\x8a\x9f\x9f\xa4fv\xe1?\xeb~\xe5AT5\x9f?l\xce\xfc\xc0\x1e\x8f\xb5?wS\xf4KuB\xc7\xbf\xe2\xe3\xadOhN\x91?\xd1\x93\xe0ZeC\xd6?\xee=\xf3j\x10r\xc4?\xa1\xa4\xf8b+\x9a\xd8?\x14/\x08[\xf4\x86\xd7\xbfO.\xfa\xa7v\x18\xbc\xbfv\xe4\x1c\x9e\x89t\xd3\xbf[\xd6\x04\x17%*\xda\xbf\xd5\xc1q\xe8I\xe5\xe0\xbfoL\xcc\xe2\x13\x82\xe8?\x83\xa9\xb5b\x1eI\xe0?\xeevhn\xb4\xa1\xd0\xbf\to\xa3,\xf1\xcb\xc7\xbf5\xde\x05\x1c\xf5\xb4\xc3?\x8e;5\x8c\xe0\x1f\xc5?\x08\xd4\x93)\xe5\xa4\xd6\xbff\xab\x83\x7f]\x98\xe9\xbf\xa7\xdd\xad\xe7\xbd\xf0\xda\xbf\x00\xed\x81\xc5\xf0\xa9\xe4\xbf"{\x12\x92\xa0\xb9\xe6?\xe1\xa1\xda\xc2\x16\x98\xe7?\x85\xd1\x90\xb9]m\xd6?\xc9\x08\x8d\x9a9B\x90?\xa5\x98\xce\x00n\xe3\xd6\xbf\xf3\x18\xa1\t\x9eR\xd1?\xf2\x9f\xcfA\x97\xfd\xe4?B\xf5^\x9f\xf4e\xb4?\x06\x85\xb8n\xbax\xd0\xbf\xd1\xd6v\x88\xb0/\xdf\xbfN\xf8\x88\x11\x86\xd1\xc9?C\xc4\xb0\x0e\xd7c\xab\xbf\x1c\x9d,\xf7\xed5\xdb?q\xa8\xaa8\xa4\xbf\xde\xbf\x19`\x01\x91\xb6\x7f\xda\xbf\xa7A\xb0g<\x12\x92\xbf\x91\x0b\x91\xbez\xc8\x9a\xbf\x1f/w\xb4\x05\x19\xcf\xbf\x14\xf4\x9e\xcd\xc6\xac\x9f\xbfF\x85\xebwS\x93\xe4?P$,#o\x10\xd0\xbf\x89\x19\xa0\xb9\xea~\xe1\xbf>\x07\xe2;\x17\xe4\xc2?I)\x8f\x18\xf0\xb8\xc5\xbf\xa6\xd3\xb3{\xec[\xd7?\xe8\n\xf3\x0bmm\xe5\xbf\x03\xaf\x81epy\xe8\xbf\x12\xb0\x11\xaa\x97\xae\x93\xbf\x80\xbb\xe1\x15q,\xdb?\x8c\xf9\xd9\xee\t\x9e\xda? \xbblN@,\xc2?\xac\xc9\xc9\xb6\x9de\xc8\xbf\xa1^H\xaf\\\xd1\xaa?\xfeM\xde\xec8\x1b\x83?\xf2\x9a\xfd\x0b\\D\xc3\xbf\x8cy\xf9\x0f<\xef\xc9\xbfL\xb5\xc3\x18;\x84\xaa?~\xed>\x03g\xf0\xc4\xbf?Xr1\x95j\xcc\xbf\xfar\x9f\xb3\xd8\xb2\xc2\xbf\xc3\x80{\xc8\xdf\xa6\xdc?\x1dn\x80\xacT\x94\xe7?\x8fI\xdd\x03\xf0w\xd7?1\xf4\xe4\x13#\xc8\xe3?\xaf\x94\xfbG\xf3\xe3\xb6?\x13\x03\xf4\xe3`\xa5\xd6\xbf\xca\xa4\xe4\x89\n\x12\xd0?\xe0k\xb93\xff\x87\xda?\xbbGm\xac\xa8\x89\xe4?\xe3&\xa8@\x85*\xe3?Q\xfb\x0c\xc5>\xec\xe3\xbf\x9d\xe4KG:H\xe2\xbfZ\\:\x18\xe05\xc5?|\xad\xda\xa0\xedz\xa9?\xe6\xe2\xbd\x9aP%\xd9?/\xae\xeb\xad$\x10\x93?\xf8,-\xa2\x9eO\xc9?\xac\xa6\xf0L-\x8f\xd6\xbf\xdf\xea\xab\xee\x1c#\xe5?\xcf;\xcf\x86\xc9\x8c\xec\xbfKS\xe8Xb\xbe\xe7?\xa1\xb5\xcd\xf4\xfc\xad\xd8?VN\xe9\xbd\xb8o\xe7\xbf\xcaH\xea\x1cq\x89\xcb?\xd7_z,6\xfb\xd7\xbf\xbcCGP\x06\xb8\xe8?\xbcJ\xcb\xd6Z\xcc\xd7?w\x84\xd43\\%\xd4\xbf\xb9\x1clk\xe7g\xdf\xbfc\xb4\x0b;\x10]C\xbf\x99N=\x9bx`\xc6?!\x12\xe6\x00\xc3#\xd5\xbf<Y\xc8\xd8\xb6\xf0\xc1?\x10xZ\x95x=\xe3?7C\xc0,@O\xd2?\xf0F)Z\x05$\x91\xbf\x9c\xa0-e<\x04\xc5\xbf\xdd`\x19\xc5\x87\x0b\xa9?\xf5\x96btw\xaf\xa2\xbfF\x12zO\xa3\x85\xd1\xbf\xcf\xb7\xc2\x93\xc8\xfe\xdb?\x7f[:Vk\x02\xdf?\x9b\xaf\xf8\x07z\xba\xa3\xbf\xc6V\x8e\xe4\x15s\xe1\xbf\xb0\x8c\xadI\xccf\xc5\xbf\xf1\x1d(S\x0e\x83\xd2\xbf9\xc2A\xd0\x88\x9f\xb1\xbf\xb3R\xcf\xc6\xe4$\xd7\xbf\xfb\xa6\xd3i\x9dpq\xbfS\xe3X,\xda\xd7\xde\xbfzhG@=;\xd3?\x01)w\xf3\xbf\x14\xd8?\xe3\x96q\xb6\xd8k\xcb?c"\xb5\xe7\xf3\x0b\xb0\xbf\x15UqFWA\xea?\xc7\xf0t\x17*j\xe6\xbf-\x88=x8\xf4\xe3\xbf\x9fS\x19\xca\xa0\x9e\xe6\xbf\xc2E\xbdW0W\xcb?\xc3\xbe\x1cPz\x00\xc0\xbf\xa9\xa9\xbb\x0f\xa0\xe7\xc4?\xd3\xae\x8b\xb3i\xc4\xdb?\xc3\xa1\xff\x84\xb3>\xde?]\x1c\xf3~\xd3\xb2\xd2?o\xbc\x84o\x0er\xd1?\xfb\xe5\xe4\xdd\x13\xf4\xe3\xbf\x92\xbc*s\xe3\xde\xd3\xbf\xbc.\xe3\xca\xdd\x0f\xa5?\xe1\'\x89]\x1c\xcb\xd8\xbf\xcd\xb0y\x1e\xc0\xb8\xe3\xbfq\x8b^\xddi\r\xa4?\xcf\x9d\xff\x00\xd8\x17\xa9\xbf\x15\x959H\xbf\xfb\xe4\xbfl\x9f\xde\x94T\xa9\xc3\xbf-3\xcb\xd4\xccz\xd0\xbf7\r\xfae\xa3\x16\xe1?\xf1V/\x7f\xb6\x9e\xd9?g\x89e\xe1\xa3\x8a\xa3?\xd1\xbf\xab]\xab\x80\xa0?B\x8a\xab5*\x94\xd9\xbfFS\xcf\xf5\x90\xf5\xe0?\xe4Yx\x80\x17\xa9\xdf?r\x17\xd0Co.\xd3\xbf\x8e\n\n\xcb\x12z\x7f?s\x170\r\xa8\xce\xb4?IWa\xca%\xc8\xbf?\xd1o\x18c\xe6M\xd2?s\x14^\xaa=\x82\xdc\xbf{\t\xa36O\xe3\xd6?E\xe2\xc7`\xdc\xbb\xbe?u\xc2\xe7\x14\x81\xc9\xc3\xbf\xa8\x1b\xb1o\x82q\xcc?x\x04\xfe\xb3n\x13\xb9?\x9e\x82>\xcb\xc8\xa2\xd7\xbf\xa2\x0e\xbb8\xab{\xc2\xbf\xd8\x96\x98\xee\x89\xec\xb4?h\xa9&\xf3\x80\x90\xb0\xbf\xdeF\xe8\xbb\xa0N\xd4\xbfM\xbe1\xc6\xbb~\xc4\xbf\xaa\xbc\xea\xc3\xd5|\xc7?\x02\xeb\xc4UOG\xa8?\x90\x1bK\x1c\x88O\xd1\xbf9\xe8\x04\xa5\xc2\xac\xbd\xbf\xfc\xea\xa0\xe0?\xdc\xd2?\xad\x7f\xd7.\x04\x88\xdf?!\x16;\x87\xfa-\xd3\xbfKV5i\x98\x93\xd2\xbffR\xdd\xec\x17{\xbc\xbf\x83\x1c\xda\xe5 \xfa\xcc\xbf\xa2\xc6\xac/K\x12\xe3\xbf\x96\xf4\x01\x0b\n\x05\xe3?H\xc2c\xe7\xb6\xfd\xe1?\x97TN\xdfc@\xda\xbf\xb5B\xff\xa5c\xea\xa4\xbf\xec\xda\x15\x12\xa6\x92\xb3\xbfl\xb2\x8f\xab\xb7\x98\xc1\xbfR\xa1\xd2G\x04\xb7\xd1\xbf"\x0b\xf1C\x12R\xa6\xbfSx\x7f\xa8\x1c\xf8\xe1?4\xd4`Dhd\xda?$\xa1s\xf6\x9c\xb4\xd5\xbfn\x96jFZm\xe3\xbf\x1f\x91\xef\xaa\xb6v\xde\xbf\xa1l\xdb\x17\xce\x07\xd6?\xbfQ\x84\'F\xd4\xb7\xbf\x82\x03g\x14eq\xad?L\xe5\x9c\t}X\xc4\xbf\x00S\x07\xcdP7\xe0?{\xf7n\xdc\xc9\xf1\xb3?!\xc7\x8ep\xdc\xa9\xd7\xbf\xbf#\x05\x94\xffI\xdc\xbf\x15\x8d\xce*y4\xe2\xbf\x17\xa3\xaf\xd1\xca[\xbe?Q\x05\x8e\x04\xb77\x99\xbf\x95\xb2}\x81X\x96\xd1\xbf@\xd9Ud\x98\x97\xbf?\x8fqGqRr\xdb?W\x9fy\xa2\x1f\xe4\x94?\xcd\x8f\xd8z\xd3\xa4\xe0?\x083\xc0d\xea9\xdb?fM\xadH\x00\xf6\xbd\xbf3\xf2W\xdfep\xd1\xbf\xc169\x07\xee\xbe\xc6?\xcf\xc5\x07F\xf2\x10\xd8\xbf\xfcI\'}a\xb3\xd2\xbf\x91\xffqN\x17\xc1\xa8?\xa9(\xd5\x90C\xf0\xe2?03\x07\xed\x86\xee\xcd?\xc4\x9b\x1e0\x0cN\xcd\xbf\x17\xf7\x80^\xb7\x95\xc2?\xdb\xc0(\xc2\x8a\x91\x7f?&c\xfc`\xda:\xe7?\x0b\xe1x\xcd!\x04\xe5?\xcc\xc7\xa3Q\xa8l\xdc\xbf\r:QPG@\xe4?3h\x95P\x82\x97\xd9\xbfdv\xdc\x92\xac\xcb\xdb\xbf^\xae9N\x83\xe0\xb0\xbf\x0b\xf7"\x85\xd6J\xdb?\xa3%]\t\xc9v\xd3\xbf\xcd?C\xa4\x00\xde\xc0?f1V\x0cg\x1a\xe2?p1^\xda|>\xb8?E\xc2\xa5\xcd\\\x80\xc2\xbf\xd2.\xa2\x93\xfd\x94\xe5\xbf\x98=`K\xad;\xd1?\x8f\x13\xd7\xfb\xf6q\x86\xbf\x0c\xaf\xf82]>\xc8\xbf|C\x9bb1a\xd1\xbf\x13\x02f\x7f_\xfa\xcb\xbf{7\xed\x8a6(\xcc?\xd1\x8eo\xf3\x93\x93\xb0?\xa8,\xf5\xc4[\xd7\xc5\xbf\xc4\xc8\xda\xa7m\x9a\xca?\xb2ZZhk\x0c\xe3\xbf$\x02\xcc\xccP\x9d\xd4?k\xf6Q\xcf\x90\x8c\xd4?\xc4\xdf\x1a\x90Y\\\xc8?D\xd9\x8c\x84\xf1 \xd5?\xb4\xd6\x94\xcc\xfb}\xe8?I\x0bj2=\xfc\xd1\xbf\xe1\x9e"W\x9b\x87\xcd\xbf\xd5\xe2!\x92\\=\xe0?\x81&\xca\xd2q\r\xcb?\xdee\xc5\xd9a\x81\xac?\xfc\xe3\x06\xc6\xec\xc1\x99?\xb2\xcdT^\x83q\xde??\xc4\x9e\xb5\x04"\xb0?\xa4\x0e\x127\x00q\xdb\xbf\xc3Udl\x02\xdd\xde\xbf\x058\xf6\xb88~\xd4?\x7foB\xa1\xb2\xc3\xce?M\xd5\xbe\xc3/6\xc8\xbf\xab\xbc^c\xf8\xd5\x9e\xbfX\xf1\x8aL?\xfb\xae?o\xb8\xac\xdd\xb0)\x9c\xbf0\xc7-\xd6@\x15\xe2\xbf%#V\xe1e\xec\xd6?\x056\x90\x121\x8c\xc8?\xf9\r\x9c\x99\x91\xf7\xd3?\x99\x0f\x88\xb7\xab\xb6\xd2\xbf#<\xf6B\x8c\xaa\x95?\x898\xb2\xab\x1d\x88\xd0\xbf\x91j\xad\xa9\xb9w\xdb\xbf\xbaA\x85\xc4\x1a\x06\xc9?P\xdd\x8fz.\xc8\xa7\xbf\xf1\x82T\xe6\xf7\xe8\xd8\xbf\xaf\xa2\x99y\xb9\xb9\xe1?\x1f\xaa\xa9\x03\x93\xcc\xe6?\x93@\xeb\x02j\xa8\xe2?\x8c>\x91)Z\x0f\xdd\xbf\x15\x9b\x1f\xdc\x05\n\xe9\xbf=\xbc\xd2\xc8\xa99\xd5\xbf\x8dF\x9f\xe5\xe3\x80\xe5?\x06\x02\x9cL\xdd\xeb\xa6?\xde9\x08_E{\xb6?gz\xe7x\xb4\xfb\xc7\xbf\x9c6\xc7m5B\xd1?s\x9e\x04j\xac\x1f\xdd?a\xba\x01\x0f\tf\xe0\xbf\xbe7\xc0 \xab\xae\xe3?\xdf\xf0B\x14\xffU\xe4?\xed\xb2\x05e\xc4y\xd3\xbf\xd9\x01\x92\x92\xe1L\xdf?3<w\x10n\x12\xe5\xbfL]\xf1U\x90\x86\xda?\xb6c\xa2\x0e\x94\t\xe5?\x10\xe8g\xa2B\n\x96\xbf\x93l^\xe8\x8cR\xd0?\xf3\x83\x8e\x96\xb3j\xd3?w\xce\x13!@\\\xb3?}Gw\x17L\x1d\xe0\xbf\xab\xcf\xbf\xea\xea\x84\xd6?WO\x1a\x0cO\xf4\xd0\xbfv\xf98\x18\xe7\t\xdc?\xe9)\x0cm\xbc\x91\xcb\xbf\xcb1\xe31jV\xdd\xbf(\x02\x87\xef\xba\x8d\xbf\xbfjR\xf8\x98\xf3.\xcd\xbf\x03\x8a.\x06\xcb\x93\xd5?\'\xff\xc7\xa4?\xbd\xdb\xbf\xef\xd7a\xa9\x8d/\xd1\xbf=\xaa\xc3+\x90\xbd\xaa\xbf\xc9_\xbc\xfbU\x84\xe1?\xfbC\xb1}~s\xd1?\xf4\xed\x03qL\xdd\xe6\xbf\xa2\x99\xaa!\x1fE\xe6\xbf\xf0\x94\xff\x7f\x147\xb4?m[\x17w-O\xb2\xbfX.J\x0f\x12\xc8\xc8?9\xa5l[\x05\xec\xc1?I(\x81W \x9e\xe3?\xe6T\'HoI\xdd?\xa1\xf6\x8b\x90\x00\xef\xc6?V\x8f\xca;{\xdf\xe2?\xb4\xe0gq\x08\x90\xe9\xbf\xf2\xb8\xaf\xbc\xf6D\xe4\xbf\xec`$\xa8\x02\x8a\xc9\xbf\x85\x16\xfeE\xfc^\x9e?\xe52\x06\xdd\xdab\xd6?\xf4\xcd\xab ]\xf8\xdb\xbf\xe9uf\xeeU\xfe\xba?{\xcb\xe7\xf2\xe0N\xba\xbf\xa4\x9b\xdex\x94x\xb1?h\x1a\x01\xf7\x03e\xd6?\xd7\x81\xff&\xfaJ\xd1\xbf\xeb{\x16dq(\xc5?\x04\xe8\xf4\xef/\xce\xba\xbf\xe5\x9e\xaa\\\x83\x07R?\x8e\x11yJN\x15\xdd?r\x84\x86\x04\x1f\xe6\xdd\xbf-u\x15\xaaj_\xde\xbf\x8b$\xb2\x82\x82\xc5\x83?\x19\xb8JU\x90\x14\xdc?\xfa\xa5\xc0\xa5\x1eA\x9d\xbf\x02\xf7\x92\x04\x9c\xe9\xe3\xbfY\xe9\n\xaeV\x81\xbe\xbf\xd5\xa2]\x11\xcb\xab\xcf?\x07F\x04@\xd2\xcc\xa0\xbf\xbc\xb7\xc6\xe9\xbe\x0b\xc7\xbf\xa4Z\x93\xe8,\xb3\xe4?\xf4SG\x02\x9a\xfd\xee?U\xe1\xb8\x1a\xe9J\xe2?2\xe8\x82X\xd5\x93\xd8\xbf\xf58k\r\xcd\xad\xaa?\\\x00]\xb6\x07\xe3\xe8\xbf\xb15\x7f.\xbd\xe4\xc8\xbfv\xcb\xda \xffy\xcf\xbfG\x1ai\xd4\x80~\xdb?l\xe0\x89?\x05\x86\xc2\xbf\xaa\x0f\x02\xcc\x9c(\xd7?.\x0cv\x87Y&\xe9?\xbb\x86a\xb6\x1d\xb2\xa3?0\xd8/\x14T\xc9\xe5?\xc6\xa9\xd9fd\x12\xe3?7:\xf3!Em\xbb\xbf9o\x87\xddr9\xdb\xbf\xdbv\xc7\xbd\x7f\x8e\xb4?`\x16\xa5t\xa9\x83\xd2\xbfhc\x02\xc9\xda\x85\xdc?\xec\xb0\x17i\xc3$\xda\xbf$\x052C\n\xcd\xd6\xbf^\xfd\x9a\xb6\xac \xdb?\xaa\xae%\x06M\xeb\xe3?\xa3\x0e\xdc\xac\x03 \xe2?\xa4\xe3\xf13\xf08\xe2\xbf\xdeO\x93\xd0\xa3\xe5\xcd?\xd9\xef+%\xf6H\xe6\xbf)\xa0\xca\xff\xe7>\xd3?!Oqy\xb4C\x8a?\xe4-M?V1\xe5?X\x03\xcf\x15\x1a\x19\xe7?a\xaf9((\xab\xda\xbfYr\xa9{\x7fl\xd5\xbf\xe2!Zs\xd1=\xa1?\xdf\xc7\x0b_\x06W\xde?\xa8e]\xc4q\xd0\xe0?\x94I7?if\xd2?b=\xb0\xb6Tt\xd1?n\x8c\xda5\xa4\xf9\xe7\xbf\xfa\xbf\x01\x97Y\x8e\xd6??\xd5\xb9~\xfe\xe3\xea?\x99I\xdd\x84\xfc*\xe3\xbfI:\'D\xa7\xce\x92?@~\x112\xb3\n\xe8\xbf!\xb3\xb4\xa3}\x9a\xe6?[\x13tl/\xb2\xe7?\xbb\xaax\xa9\xb7\x8e\xd4?\xef\x04\x11n*7\xc0\xbf\x84\x8c\x1bY\x04\x88\xa9\xbf;\x9c\x86\x9a\xfc\x96\xd0\xbf\xfe\xb9\xd4q\x82\x0f\xe0\xbf(\xa3\xd1F\xce\x00\xe1?K\x91Ba\xc8\x95\xd5?\xb7m\x1f\xce\x86(\xc0?\xa0\x05X\xe2\xef\xd5\xbe\xbf\xefR\xcb\xe72\x95\xc7?)\x18\xd8\x84\xdc\x0b\xa7?/ix@P-\xc7?\x1b"\xd3\x8b\xd3\xe7\xc0\xbf\xb2>\x82\x9dX\xc2\xc2?"j|\xf1^n\xb7\xbfDs\x13\x1eJq\xd6\xbf\x9a\xc6\x1e$\xb1\xa6\xe9?\xed\xa9cO\x92T\xe4?~\xa5(\xd3\x11N\xcb\xbf\x03\x90\x06\x06a.\xce\xbf8\xb63A\xef\x93\xdc\xbf\x84\xe2\xb8\xf1ei\xbf?\xddh\xf8"\xef\x19\xc1?\xb32L\xba\xe2\x14\xb6\xbfvDd\n\x95\x1f\xcb?[q\x18\x1a\xfc2\xd6?>>\xb2k\x06@\xcd?\xcd\x06&\xf3\xf6{\xbc?;d\x0b\xd6\x83\x90\xda\xbf\x12.6\x94\x8ag\xd3\xbf\x05_MY\x98\x03\xcb\xbf)\xd4\xb8\x9a\x95\xd2\xd2?\xf1\xfc\x88\xb88\xaf\xd7\xbf\xbe\t%\xfd4s\xd2?\x86h<,o5\xe1?\x10\xa0\xb9\xc2\x03+\xd6?ix\xa5H\xdc\xad\xe8?\x1b\xb3\xe8/\xf8\x9e\xeb?\xca42\xf6\xab@\xdf\xbf\\\xfeP~;w\xde\xbf\x0fUV\x0b\xf7\x98\xdb?Zs\x1cahQ\xca\xbf\x10\xa9TK\x10\xc7\xd5?4\x01L]O\x99\xe7\xbf.;\xc33~9\xb5\xbf\x10\x82[\xbb\xc5 \xb3\xbf\xc4(\xe2\\Rl\xcd\xbf\xef\xdd\x1d\xacdC\xe2\xbfz\xb3\x16\xef%\x8c\xc6?gv!\t\xa7^\xe6?^G\x01z\x8b4\x90\xbf$\x9d{\x114 \xc8\xbf$\x136\xbc\x84i\xb4?T\xe9\xd6\xc3d\x89\xdd?t\xf5?M_?\xd0?\x99hMm(r\xb4\xbf\x13\xe9\xdce\x97W\xc5?\x90\xb8\n\xb2\xea\xa4\xe8?r\xdc\xef\xf7\xb0\xca\xe6\xbf-\x15\xbdw\x985\xe5\xbf\x83\x94$\xce\rl\xcd\xbfH\x0c\xbf\x0cPJ\xc4?\x8aF2\xee\xd2\x91\xc2?-g\xb4\x95\xe2f\xd7\xbf\xe3\xa2\xec\x05\x81\r\xc5?\x93\x9e*\xdf.\xab\xce?\x16\xbd\x17\x8eM|\xe6\xbf5\xdd\x91\xbe\xdf\xe4\xe4\xbf\x851\xb8u\xc3\xe3\xdb?\xec\x91B\xf1\xa9\xf1\xbd\xbf\xa8\x11/\x1e\x0cz\xbe?\x1e\x12\x8c\t\x0eK\x9e?\x873V\xee\x1aA\xe1?\xfc\x0c\xbb\x86\x96@\xe0\xbf\xa1\xa7thV$\xd5\xbf\xf7\xd1\xd6\xf5\xfd\xa2\xd6\xbf\x95_\x8a\x874M\xc2\xbf\r\xc50;\x93\x13\xa7?\xde[\x84}{\x80m?\xf1\xb5\xb8\x19\x11\x81\xd1\xbf\xf4\xc9,i\xa5\x86\xd7\xbf\xa5:,\x8c1n\xe1\xbfR\xec\xf3-\x91\xc9\xc5\xbf\xc3I\x87\x1fO\r\xe1?\x10\x15\x82\xfa\xc8f\xbe?.eQ\xaa\x1d\xbc\xd1?\xfd\x91ORw\x05\xcf?P\x94:7z`\xe2?\x1e9\xdb\xa2\xbe\\\xcc\xbf#3\xf3\x94|\x9a\xe6\xbf*S\xf6\x0bkV\xdb?\xd1+:&\xac4\xdd\xbf&\xafZN\x0c\xa5\xd1?\xaa&\x94\xa3\xac\x1e\xe3\xbf\x9av\xa1\xb2\x83\r\xcd\xbf\xae\xb5\x93\x0c\xb1\xae\xd4\xbf\x98\xf5Ul,\x8a\xaf?b\x1c\xe3\xf21\xc2\xd1?5h\x0e\x05\xf0\x02\xc6\xbf\xa4`\xb1]\x13\xcc\xd4\xbf\xd7\x03\x08\x11\x0e\xf8\xe3\xbf\xfb\xc4\xc9Q\xe2`\xd2?\xac\xbd\xde_\xd9\xa9\xe5\xbfxbM\\\xf7X\xe4?V\xbf\x9ab=\xa2\xe3?8M\x94g\xafO\xd4\xbfB<\xb0\x18\xcbi\xe1?\x05\xc9~\r\x07\x01\xe7\xbf\r;\x1e\xc4\x18\xbb\xe6?\x8bU\xcd\xdbgO\xe6?\x10\xf4\x884\x93r\xa4?\x99\xe9MC.\x02\xd3?\x93\x176\xea\xf3E\xe5?\x90&\x13\x82\xc0(e\xbf\x0b\x7f\xa0[\xfe\x88\xd6\xbf\xa0/_[=n\xdc\xbf\xc0\x89\xe5=\xef\x0f\xd3?\x08\xcdj\x1a\xa9\'\xd5\xbf\x184f\xf7\xa4\xed\xe2?*\x86=\x18\xd82\xd7?\x9b_\xf9\xa6j\x00\xbe\xbf\xbcG\x00\xb1\x00\xf3\xcd?\x1b\xddciHt\xde?\xbe\x0b\xaa\x95\xd3<\xe3\xbf\xdd\x00\xc6bZ-\xe8\xbf\xcf\'\xd5\xe7\\J\x99?-\x19\xc91\x1a\xcd\xcf\xbfN\x9a\xaf\xa1E\xb5\xc1\xbf\r\x91\xb4\x07\x93\x0b\xc4?\xa4$\x11n\xcd[\xe4?\xd4]\x94b\x84b\xcb?2\x04\x02\xa5D\x1a\xb0?\xc4\x01Bf\x01*\x98\xbf0\xc9{\xd3\xaf\x97\xde\xbf\x9f\x7f\xe9C\xe3\xea\xeb\xbfx\xa47y\xf2|\xe8?\xa7*[\xf0\x9cq\xc0\xbf\xa1X\xe8\xa0\xce\x13\xdc\xbf\xe4igq\xaf"\xd6\xbf\xde\x86\xc5\xbaQ\xe2\xbc?*O\xc7d\x00\xb5\xe2?{;n\xf8\xddg\xce?\x06\xb9\x95\xc6Y\xff\xea?\xda\xb9\x07\xc2\xa0\x02\xe8\xbf\xafK\xf0\x07S7\xe9\xbf\xb6\xe2\x8e#\x7f\x0f\xdb\xbf4h\x99-\xa3\xe7Z\xbf\x17\xd7\x95<\x89\x06\xea\xbf\xa2\x17U5\xf2\xb6\xdd?Er\x7f\xf7\xe2L\xe6?Kf\xf1h\x93\x87\xb9?\x1b\xa8O|\xf5\xf1\xad?\xf2\xda\x17D\xca\xff\xc6?\xb8F\xfbyWC\xe3\xbff,\x85N\xfc\xef\xe1\xbf\x10\xdb\xc25\xf9\x13\xcf? \x9b\xd8o\xdb\x9e\xdd?\xf1Mj\xd4\xa9w\xde\xbf\x8c\x96\xe9-\xea|\xd5?\x12\x93\xca\x97\xec\xe9\xda?`\xf0"\x19l)\xd1\xbfN\x96\x81\x91\xee\'\xd4?8[\x8d\x96_\x88\xe4?l\xb7"\xcd\xc2\xe7\xe0\xbf\xc5W\xe1\x0e\x96\x19\xe8\xbf#i\x03\x95\x05\xf3\xe3?%\n\xef\xe1\xb8{\xd2\xbfgDc\xe1\x08\xbb\xd0\xbf\xe7\x13y\x96}J\xd4\xbf[\x06\xd1\xa1\xd8\x81\xe4\xbf#\r\xa6&\x06\xd9\xd2?\xeb\x87\x92>\xa6\xfc6?\xff\xe0\xbe\xaeG\x98\xd3?o\xec\x13\xe2\x94E\xa1\xbf\xd6\x1d\\\x1fp\x9b\xe0\xbfd\xc5bK8\x10\xaf?\x0f\x03\xc3\n!C\xdf\xbf\x82\xe5\x8fN\x05b\xa7?F\xdfi\x9bqY\xd3\xbfPQkO\xc74\xda\xbf\xac\xd4F\xcc\xa6\xcd\xd8\xbf|\x1f\xa8\x0bP\xcd\xde\xbf\xc1\x1b\xc4\xa7\xddN\xc1?]\x05D\xcc\xc4\x9f\xda\xbf,(\xb8R\x04\xa3\xe6\xbfrh\x8d\x83\xf3\x89\xd5?\x95l\xec\x89\xf4\x90\xd1\xbf\xfa\x05\xec\xc8\xe6\x16\xcf\xbfQ\xe4\x11\x89\xd0\xb5\xd1?M\xa6y:g\xf3\xdd?\x8d\xed\xbe\x97\xeb\x1f\xc9?\xfbA\xa5\x85%\xa7\xd4?\x12J\xad\xbb%s\xe0\xbf\xdc\xf5; \x98\xbc\xbf\xbf\xba9\x91\x10}X\xde?\xf1.\x1bEFQ\xe4\xbf4\xc7j\xfd\x1fp\xdb?T\xban\xd4\x19;\xbd?\xa5\x88Z\xc3\xed\xd1\xd2?\xf3_\x83\x91M\xc3\xda?\xa5\xb6{\x1fl\xd4\xcd\xbfM\xd7t\xdb\xabZ\xd2\xbfE\xa1\x90\xec\x07*\xdc?t\xa8\xbf\x14\x0c\x08\xc3?\xfa\xd1\xcd\xd6d\x95\xe5\xbf\xce\x99>\xf6\xb3a\xe4?\x03\xadf\xe9+/\xd2\xbf\x0b4\xdf\xc7\x00\xb3\xc7\xbf\xe5\x15P\xc4m\x83\xc9\xbf\x90_uy\xd0\x0c\xae?\x12\x06\x01e\xbbH\xe1?\x81I\xe8\x99Af\xcc\xbf\x12\xd2{\xca\xe7c\xc6\xbf$WL#$\xce\xe9\xbf\xea\xa2\x89\x10\x90<\xd0?t\xed`\x98o8\xdd\xbf\xeeb~-\xee\xd0\xc9\xbf\xb8\x04\xe6]\x7fJ\xdd\xbf`3+\xbf&\xa3\xdf?\xc7\x8b\xf7]\xc5\x05\xa8\xbf\xaf[\x8f8j\xb7\xca\xbf\xce/\xe0N/h\xa0\xbf\x98c\xb2V\x0b\xdc\xe4?\xd7q\xf8q\x058\xe7\xbf\xc6\x93k\xa3n\xa6\xe9\xbf\xa5shape\x92\xcc\xc8\x05\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0')
partial_utility_solution = pd.read_msgpack(b'\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\xc8\x00D\x00\x00\x00\x00\x00\x00\x00i\x00\x00\x00\x00\x00\x00\x00\xb1\x00\x00\x00\x00\x00\x00\x00\xb6\x00\x00\x00\x00\x00\x00\x00\xe8\x00\x00\x00\x00\x00\x00\x00\xf9\x00\x00\x00\x00\x00\x00\x00\x12\x01\x00\x00\x00\x00\x00\x00 \x01\x00\x00\x00\x00\x00\x00*\x01\x00\x00\x00\x00\x00\x003\x01\x00\x00\x00\x00\x00\x00>\x01\x00\x00\x00\x00\x00\x00|\x01\x00\x00\x00\x00\x00\x00\x83\x01\x00\x00\x00\x00\x00\x00\x9e\x01\x00\x00\x00\x00\x00\x00\xc0\x01\x00\x00\x00\x00\x00\x00\xda\x01\x00\x00\x00\x00\x00\x00\xe0\x01\x00\x00\x00\x00\x00\x00\xe3\x01\x00\x00\x00\x00\x00\x00N\x02\x00\x00\x00\x00\x00\x00W\x02\x00\x00\x00\x00\x00\x00X\x02\x00\x00\x00\x00\x00\x00[\x02\x00\x00\x00\x00\x00\x00^\x02\x00\x00\x00\x00\x00\x00`\x02\x00\x00\x00\x00\x00\x00b\x02\x00\x00\x00\x00\x00\x00\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7(\x00\x01\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x002\x00\x00\x00\x00\x00\x00\x00h\x00\x00\x00\x00\x00\x00\x00\x99\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x91\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x19\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc7\xc8\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\x06\x00\x00\x00\x00\x00\x00\x00\x07\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\t\x00\x00\x00\x00\x00\x00\x00\n\x00\x00\x00\x00\x00\x00\x00\x0b\x00\x00\x00\x00\x00\x00\x00\x0c\x00\x00\x00\x00\x00\x00\x00\r\x00\x00\x00\x00\x00\x00\x00\x0e\x00\x00\x00\x00\x00\x00\x00\x0f\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x00\x11\x00\x00\x00\x00\x00\x00\x00\x12\x00\x00\x00\x00\x00\x00\x00\x13\x00\x00\x00\x00\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00\x15\x00\x00\x00\x00\x00\x00\x00\x16\x00\x00\x00\x00\x00\x00\x00\x17\x00\x00\x00\x00\x00\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc8\x03\xe8\x00\x00\x00\x00\x00\x00\x00\x04@\x00\x00\x00\x00\x00\x00\x0c@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x04@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x0c@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x0c@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x0c@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x0c@\x00\x00\x00\x00\x00\x00\x0c@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x0c@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\x0c@\x00\x00\x00\x00\x00\x00\x04@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x0c@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x04@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x04@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x04@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x08@\xa5shape\x92\x19\x05\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0')

print('Check utility: ', end='')
np.testing.assert_allclose(utility.head(), partial_utility_solution, rtol=6e-1)
print('success!')

print('Check similarity: ', end='')
np.testing.assert_allclose(similarity.head(), partial_similarity_solution, rtol=6e-1)
print('success!')

### Snel werken
Zoals je merkt kost het even wat rekentijd om een similarity matrix op te bouwen. Dit kunnen we daarom beter 1x doen, even opslaan en vervolgens kunnen we dit weer inladen. Dit gaat makkelijk met `pandas`. Dat zie je ook telkens in de tests waar we `pd.read_msgpack` gebruiken. [Msgpack](https://msgpack.org/) is een dataformaat dat in weinig bytes veel informatie kwijt kan en snel ingeladen kan worden. Daarom gebruiken we dit formaat voor tests. Maar, `pandas` ondersteund veel meer formaten, ook `.csv`.

In de code hieronder slaan we de matrices op als een `.msgpack` bestand, en laden we deze vervolgens weer in om even te laten zien hoe dat werkt.

In [ ]:
utility.to_msgpack('./mini-movielens/ratings_t80_utility_student.msgpack')
similarity.to_msgpack('./mini-movielens/ratings_t80_similarity_student.msgpack')

u = pd.read_msgpack('./mini-movielens/ratings_t80_utility_student.msgpack')
s = pd.read_msgpack('./mini-movielens/ratings_t80_similarity_student.msgpack')

pd.testing.assert_frame_equal(u, utility)
pd.testing.assert_frame_equal(s, similarity)

### Voorberekende data

In de rest van de opgave maken we gebruik van een aantal verschillende datasets. Om het laden wat te versnellen en om er zeker van te zijn dat iedereen met dezelfde data rekent hebben we de data alvast gesplitst en de utility en de (adjusted cosine) similarity matrix uitgerekend. Met de onderstaande code kan je deze data laden.

In [ ]:
ratings = pd.read_csv(f'./mini-movielens/ratings_t80.csv')[['userId', 'movieId', 'rating', 'timestamp']]
training_data = pd.read_msgpack(f'./mini-movielens/ratings_t80_training.msgpack')
test_data = pd.read_msgpack(f'./mini-movielens/ratings_t80_test.msgpack')
utility_items = pd.read_msgpack(f'./mini-movielens/ratings_t80_utility_items.msgpack')
similarity_items = pd.read_msgpack(f'./mini-movielens/ratings_t80_similarity_items.msgpack')

display(utility_items.head())
display(similarity_items.head())

print(ratings.shape, training_data.shape, test_data.shape, utility_items.shape, similarity_items.shape)

### Vraag 2

Schrijf hieronder een functie die voor elke rating in de _test data_ een rating voorspelt. Gebruik hiervoor dezelfde _item based_ aanpak als in de vorige opdracht. 

De input van de functie is de similarity matrix, de utility matrix en de test data. De output is een copy van de test data met een extra kolom: de voorspelde rating. De bedoeling is dat de output er zo uitziet als hieronder: 

<table border="1" class="dataframe"><thead><tr style="text-align: right;"><th></th><th>userId</th><th>movieId</th><th>rating</th><th>predicted rating</th></tr></thead><tbody><tr><th>1</th><td>68</td><td>2</td><td>2.5</td><td>3.487983</td></tr><tr><th>3</th><td>68</td><td>47</td><td>4.0</td><td>3.457851</td></tr><tr><th>6</th><td>68</td><td>110</td><td>2.5</td><td>3.625071</td></tr><tr><th>14</th><td>68</td><td>344</td><td>2.5</td><td>3.457997</td></tr><tr><th>27</th><td>68</td><td>593</td><td>3.5</td><td>3.494299</td></tr><tr><th  style="text-align: center;" colspan=5>...</th></tr></tbody></table>

In [ ]:
def predict_ratings_item_based(similarity, utility, test_data):
    # TODO
    
    
predicted_item_based = predict_ratings_item_based(similarity_items, utility_items, test_data[['userId', 'movieId', 'rating']])
display(predicted_item_based.head())

In [ ]:
print('Computing solution: ', end='')
_student_solution = predict_ratings_item_based(similarity_items, utility_items, test_data[['userId', 'movieId', 'rating']]).head()
_partial_solution = pd.read_msgpack(b'\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xa5Index\xa4name\xc0\xa5dtype\xa6object\xa4data\x94\xa6userId\xa7movieId\xa6rating\xb0predicted rating\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7(\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x06\x00\x00\x00\x00\x00\x00\x00\x0e\x00\x00\x00\x00\x00\x00\x00\x1b\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x92\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x02\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xd8\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7P\x00\x00\x00\x00\x00\x00\x00\x04@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x04@\x00\x00\x00\x00\x00\x00\x04@\x00\x00\x00\x00\x00\x00\x0c@`\xa4({c\xe7\x0b@\xba\x97T\xaf\xad\xa9\x0b@\x99\x137j%\x00\r@c\xd6\xf1\x88\xfa\xa9\x0b@\xbc4\x87\xe8R\xf4\x0b@\xa5shape\x92\x02\x05\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x02\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xd8\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7P\x00D\x00\x00\x00\x00\x00\x00\x00D\x00\x00\x00\x00\x00\x00\x00D\x00\x00\x00\x00\x00\x00\x00D\x00\x00\x00\x00\x00\x00\x00D\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00/\x00\x00\x00\x00\x00\x00\x00n\x00\x00\x00\x00\x00\x00\x00X\x01\x00\x00\x00\x00\x00\x00Q\x02\x00\x00\x00\x00\x00\x00\xa5shape\x92\x02\x05\xa5dtype\xa5int64\xa5klass\xa8IntBlock\xa8compress\xc0')
print('success!')

print('Testing layout of prediction: ', end='')
for column in ['userId', 'movieId', 'rating', 'predicted rating']:
    assert column in _student_solution.columns, f'expected column {column} in output'

print('success!')

print('Testing values of prediction: ', end = '')
np.testing.assert_allclose(_student_solution, _partial_solution, rtol=2e-1)
print('success!')

### Mean squared error

Hoe meten we nu hoe goed de voorspelling is? De manier waarop we dit doen lijkt een beetje op bepalen van _similarity_ tussen films of gebruikers. We bepalen eerst de afwijking (de _error_) per voorspelde rating:

<table border="1" class="dataframe"><thead><tr style="text-align: right;"><th>user</th><th>movie</th><th>:</th><th>actual<br>rating</th><th>-</th><th>predicted<br>rating</th><th>=</th><th>difference</th></tr></thead><tbody>
    <tr><td>68</td><td>2</td><td></td><td>2.5</td><td>-</td><td>3.487983</td><td>=</td><td>-0.99</td></tr>
    <tr><td>68</td><td>47</td><td></td><td>4.0</td><td>-</td><td>3.457851</td><td>=</td><td>0.54</td></tr>
    <tr><td>68</td><td>110</td><td></td><td>2.5</td><td>-</td><td>3.625071</td><td>=</td><td>-1.13</td></tr>
    <tr><td>68</td><td>344</td><td></td><td>2.5</td><td>-</td><td>3.457997</td><td>=</td><td>-0.96</td></tr>
    <tr><td>68</td><td>593</td><td></td><td>3.5</td><td>-</td><td>3.494299</td><td>=</td><td>0.01</td></tr>
    <tr><th  style="text-align: center;" colspan=8>...</th></tr></tbody></table>
    
Dan nemen we het gemiddelde van de kwadraten van de afwijkingen:

$$
\textrm{mean square error} = \frac{(-0.99)^2 + 0.54^2 + (-1.13)^2 + (-0.96)^2 + 0.01^2 + \ldots}{N}
$$

Waarbij $N$ het aantal items in de test set is. Deze maat voor de afwijking wordt de **mean squared error** (mse) genoemnd. Dit is een zeer veel gebruikte maat in data science en machine learning, je zal hem in de toekomst nog vaak tegenkomen. 

De formele definitie voor de liefhebber:

$$
\textrm{mse} = \frac{1}{N}\sum_{i=1}^N(a_i - p_i)^2
$$

waarbij $a_i$ de echte rating en $p_i$ de voorspelde rating voor sample $i$.

### Vraag 3

Implementeer de functie `mse()` hieronder:

In [ ]:
def mse(predicted_ratings):
    # TODO
    
    
print(mse(predicted_item_based))

In [ ]:
np.testing.assert_allclose(mse(predicted_item_based), 0.6, rtol=0.15)

### Wat betekent dit?

Als het goed is komt er een getal rond de 0.6 uit de mean squared error voor de voorspellingen voor deze specifieke dataset. Is dat goed? Het probleem met _mse_ is dat de waarde op zichzelf niet zoveel zegt. Het is vooral nuttig om uitkomsten met elkaar te kunnen vergelijken. We kunnen bijvoorbeeld kijken hoe goed _item based filtering_ het doet ten opzichte van _user based filtering_.

### Vraag 4

Implementeer hieronder de functie `predict_ratings_user_based()`. De output moet verglijkbaar zijn met de functie `predict_ratings_item_based()`:

<table border="1" class="dataframe"><thead><tr style="text-align: right;"><th></th><th>userId</th><th>movieId</th><th>rating</th><th>predicted rating</th></tr></thead><tbody><tr><th>0</th><td>68</td><td>1</td><td>2.5</td><td>3.803867</td></tr><tr><th>2</th><td>68</td><td>16</td><td>3.5</td><td>3.931587</td></tr><tr><th>9</th><td>68</td><td>231</td><td>3.5</td><td>2.789119</td></tr><tr><th>10</th><td>68</td><td>260</td><td>5.0</td><td>4.378258</td></tr><tr><th  style="text-align: center;" colspan=5>...</th></tr></tbody></table>

In [ ]:
def predict_ratings_user_based(similarity, utility, test_data):
    # TODO
    
    
test_data = pd.read_msgpack(f'./mini-movielens/ratings_t80_test.msgpack')
utility_users = pd.read_msgpack(f'./mini-movielens/ratings_t80_utility_users.msgpack')
similarity_users = pd.read_msgpack(f'./mini-movielens/ratings_t80_similarity_users.msgpack')

predicted_user_based = predict_ratings_user_based(similarity_users, utility_users, test_data[['userId', 'movieId', 'rating']])
display(predicted_user_based.head())
print(mse(predicted_user_based))

In [ ]:
print('Testing user based versus item based prediction: ', end = '')
assert mse(predicted_user_based) > mse(predicted_item_based), "expected a slightly lower error for item based prediction"
print('success!')

### De lange staart

Een bekend probleem met het soort data waar we nu mee bezig zijn is de _long tail_. Voor een groot deel van de data is er weinig informatie beschikbaar. Met andere woorden, de dichtheid van de data kan zeer laag zijn. Laten we kijken wat de invloed van de dichtheid op de data is op de kwaliteit van de voorspelling.

De folder `./mini-movielens/` bevat de datasets `ratings_t20` t/m `ratings_t80`. Deze hebben resepectievelijk een dichtheid van 20% t/m 80%. Dat wil zeggen dat bij bijvoorbeeld `ratings_t20` de gebruikers gemiddeld 20% van de films een rating hebben gegeven. Voor elke dataset is al een test en training set gemaakt en zijn de utility en similarity matrices al voor je uitgerekend. Deze hebben de uitgangen `_test.msgpack`, `_training.msgpack`, `_utility_items.msgpack`, `_utility_users.msgpack`, `_similairity_items.msgpack` en `_similairity_users.msgpack`. 

Voor de onderstaande vraag de dichtheden van de datasets wil weten. Let hierbij op dat de dichtheid van de trainingset lager is dan die voor de volledige dataset. Daar zal je rekening mee moeten houden. Je kan de dichtheid van een set opvragen met de functie `density()`

### Vraag 5

Maak hieronder een plot met op de x-as de dichtheid van de data en op de y-as de mse. Doe dit voor zowel de user based filtering en item based filtering in dezelfde plot. 

In [ ]:
# TODO


### Vraag 6

Bespreek (kort) de resultaten. Is er een effect van de dichtheid op de kwaliteit van de voorspelling? Is er een verschil tussen user based en item based? Hoe kunnen we dat verklaren aan de hand van de data? 

YOUR ANSWER HERE

### Tot slot

In de praktijk is het probleem vaak nog veel groter. De gemiddelde dichtheid voor de volledige MovieLens dataset is minder dan 1%. Je kan je voorstellen dat het voor de meeste populaire films nog wel lukt om aanbevelingen te doen met collaborative filtering, maar voor het grootste deel is er een ander oplossing nodig. Hier gaan we in de volgende opdracht naar kijken. 